# File "Network"

In [1]:
"""""""""""""""""""""""""""""""""""""""
network.py
Модуль создания и обучения нейронной сети для распознавания рукописных цифр
с использованием метода градиентного спуска.
Группа:ИСТб-22-1
ФИО:Моргунов Александр Алексеевич
"""""""""""""""""""""""""""""""""""""""
#### Библиотеки
# Стандартные библиотеки
import random # библиотека функций для генерации случайных значений
# Сторонние библиотеки
import numpy as np # библиотека функций для работы с матрицами
""" ---Раздел описаний--- """
""" --Описание класса Network--"""
class Network(object): # используется для описания нейронной сети
    def __init__(self, sizes): # конструктор класса
        # self – указатель на объект класса
        # sizes – список размеров слоев нейронной сети
        self.num_layers = len(sizes) # задаем количество слоев нейронной сети
        self.sizes = sizes # задаем список размеров слоев нейронной сети
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]] # задаем случайные начальные смещения
        self.weights = [np.random.randn(y, x) for x, y in zip(sizes[:-1],sizes[1:])] # задаем случайные начальные веса связей
    def feedforward(self, a):
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
        return a
    def SGD( # Стохастический градиентный спуск
    self # указатель на объект класса
    , training_data # обучающая выборка
    , epochs # количество эпох обучения
    , mini_batch_size # размер подвыборки
    , eta # скорость обучения
    , test_data # тестирующая выборка
    ):
        test_data = list(test_data) # создаем список объектов тестирующей выборки
        n_test = len(test_data) # вычисляем длину тестирующей выборки
        training_data = list(training_data) # создаем список объектов обучающей выборки
        n = len(training_data) # вычисляем размер обучающей выборки
        for j in range(epochs): # цикл по эпохам
            random.shuffle(training_data) # перемешиваем элементы обучающей выборки
            mini_batches = [training_data[k:k+mini_batch_size] for k in range(0, n, mini_batch_size)] # создаем подвыборки
            for mini_batch in mini_batches: # цикл по подвыборкам
                self.update_mini_batch(mini_batch, eta) # один шаг градиентного спуска
            print ("Epoch {0}: {1} / {2} = {3}".format(j, self.evaluate(test_data), n_test, self.evaluate(test_data)/n_test)) # смотрим прогресс в обучении
    def update_mini_batch( # Шаг градиентного спуска
        self # указатель на объект класса
        , mini_batch # подвыборка
        , eta # скорость обучения
        ):
        nabla_b = [np.zeros(b.shape) for b in self.biases] # список градиентов dC/db для каждого слоя (первоначально заполняются нулями)
        nabla_w = [np.zeros(w.shape) for w in self.weights] # список градиентов dC/dw для каждого слоя (первоначально заполняются нулями)
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y) # послойно вычисляем градиенты dC/db и dC/dw для текущего прецедента (x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)] # суммируем градиенты dC/db для различных прецедентов текущей подвыборки
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)] # суммируем градиенты dC/dw для различных прецедентов текущей подвыборки
            self.weights = [w-(eta/len(mini_batch))*nw for w, nw in zip(self.weights, nabla_w)] # обновляем все веса w нейронной сети
            self.biases = [b-(eta/len(mini_batch))*nb for b, nb in zip(self.biases, nabla_b)] # обновляем все смещения b нейронной сети
    def backprop( # Алгоритм обратного распространения
        self # указатель на объект класса
        , x # вектор входных сигналов
        , y # ожидаемый вектор выходных сигналов
        ):
        nabla_b = [np.zeros(b.shape) for b in self.biases] # список градиентов dC/db для каждого слоя (первоначально заполняются нулями)
        nabla_w = [np.zeros(w.shape) for w in self.weights] # список градиентов dC/dw для каждого слоя (первоначально заполняются нулями)
        # определение переменных
        activation = x # выходные сигналы слоя (первоначально соответствует выходным сигналам 1-го слоя или входным сигналам сети)
        activations = [x] # список выходных сигналов по всем слоям (первоначально содержит только выходные сигналы 1-го слоя)
        zs = [] # список активационных потенциалов по всем слоям (первоначально пуст)

        # прямое распространение
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b # считаем активационные потенциалы текущего слоя
            zs.append(z) # добавляем элемент (активационные потенциалы слоя) в конец списка
            activation = sigmoid(z) # считаем выходные сигналы текущего слоя, применяя сигмоидальную функцию активации к активационным потенциалам слоя
            activations.append(activation) # добавляем элемент (выходные сигналы слоя) в конец списка
        # обратное распространение
        delta = self.cost_derivative(activations[-1], y) * sigmoid_prime(zs[-1]) # считаем меру влияния нейронов выходного слоя L на величину ошибки (BP1)
        nabla_b[-1] = delta # градиент dC/db для слоя L (BP3)
        nabla_w[-1] = np.dot(delta, activations[-2].transpose()) # градиент dC/dw для слоя L (BP4)

        for l in range(2, self.num_layers):
            z = zs[-l] # активационные потенциалы l-го слоя (двигаемся по списку справа налево)
            sp = sigmoid_prime(z) # считаем сигмоидальную функцию от активационных потенциалов l-го слоя
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp # считаем меру влияния нейронов l-го слоя на величину ошибки (BP2)
            nabla_b[-l] = delta # градиент dC/db для l-го слоя (BP3)
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())# градиент dC/dw для l-го слоя (BP4)
        return (nabla_b, nabla_w)
    def evaluate(self, test_data): # Оценка прогресса в обучении
        test_results = [(np.argmax(self.feedforward(x)), y) for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)
    def cost_derivative(self, output_activations, y): # Вычисление частных производных стоимостной функции по выходным сигналам последнего слоя
        return (output_activations-y)
""" --Конец описания класса Network--"""
""" --- Конец раздела описаний--- """
""" ---Тело программы--- """
net = Network([2, 3, 1]) # создаем нейронную сеть из трех слоев
""" ---Конец тела программы--- """
""" Вывод результата на экран: """
print('Сеть net:')
print('Количетво слоев:', net.num_layers)
for i in range(net.num_layers):
    print('Количество нейронов в слое', i,':',net.sizes[i])
for i in range(net.num_layers-1):
    print('W_',i+1,':')
    print(np.round(net.weights[i],2))
    print('b_',i+1,':')
    print(np.round(net.biases[i],2))

Сеть net:
Количетво слоев: 3
Количество нейронов в слое 0 : 2
Количество нейронов в слое 1 : 3
Количество нейронов в слое 2 : 1
W_ 1 :
[[-0.79 -0.42]
 [-2.36  0.72]
 [-0.84 -0.27]]
b_ 1 :
[[0.66]
 [0.33]
 [0.9 ]]
W_ 2 :
[[ 0.71 -0.36  0.58]]
b_ 2 :
[[-1.07]]


In [2]:
def sigmoid(z): # определение сигмоидальной функции активации
    return 1.0/(1.0+np.exp(-z))

In [3]:
def sigmoid_prime(z):# Производная сигмоидальной функции
    return sigmoid(z)*(1-sigmoid(z))

# File "MNIST loader"

In [4]:
"""
mnist_loader.py
~~~~~~~~~~
Модуль для подключения и использования базы данных MNIST.
Группа:ИСТб-22-1
ФИО:Моргунов Александр Алексеевич
"""
import gzip # библиотека для сжатия и распаковки файлов gzip и gunzip.
import pickle # библиотека для сохранения и загрузки сложных объектов Python.
import numpy as np # библиотека для работы с матрицами
def load_data():
    f = gzip.open('mnist.pkl.gz', 'rb') # открываем сжатый файл gzip в двоичном режиме
    training_data, validation_data, test_data = pickle.load(f, encoding='latin1') # загружам таблицы из файла
    f.close() # закрываем файл
    return (training_data, validation_data, test_data)

In [5]:
def load_data_wrapper():
    tr_d, va_d, te_d = load_data() # инициализация наборов данных в формате MNIST
    training_inputs = [np.reshape(x, (784, 1)) for x in tr_d[0]] # преобразование массивов размера 1 на 784 к массивам размера 784 на 1
    training_results = [vectorized_result(y) for y in tr_d[1]] # представление цифр от 0 до 9 в виде массивов размера 10 на 1
    training_data = zip(training_inputs, training_results) # формируем набор обучающих данных из пар (x, y)
    validation_inputs = [np.reshape(x, (784, 1)) for x in va_d[0]] # преобразование массивов размера 1 на 784 к массивам размера 784 на 1
    validation_data = zip(validation_inputs, va_d[1]) # формируем набор данных проверки из пар (x, y)
    test_inputs = [np.reshape(x, (784, 1)) for x in te_d[0]] # преобразование массивов размера 1 на 784 к массивам размера 784 на 1
    test_data = zip(test_inputs, te_d[1]) # формируем набор тестовых данных из пар (x, y)
    return (training_data, validation_data, test_data)

In [6]:
def vectorized_result(j):
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e

# Interpretator

In [10]:
training_data, validation_data, test_data = load_data_wrapper()

In [11]:
net = Network([784, 30, 10])

In [12]:
net.SGD(training_data, 30, 10, 3.0, test_data=test_data)

Epoch 0: 8618 / 10000 = 0.8618
Epoch 1: 8689 / 10000 = 0.8689
Epoch 2: 8823 / 10000 = 0.8823
Epoch 3: 8903 / 10000 = 0.8903
Epoch 4: 9037 / 10000 = 0.9037
Epoch 5: 9026 / 10000 = 0.9026
Epoch 6: 9087 / 10000 = 0.9087
Epoch 7: 9081 / 10000 = 0.9081
Epoch 8: 9087 / 10000 = 0.9087
Epoch 9: 9094 / 10000 = 0.9094
Epoch 10: 9127 / 10000 = 0.9127
Epoch 11: 9137 / 10000 = 0.9137
Epoch 12: 9184 / 10000 = 0.9184
Epoch 13: 9153 / 10000 = 0.9153
Epoch 14: 9153 / 10000 = 0.9153
Epoch 15: 9208 / 10000 = 0.9208
Epoch 16: 9119 / 10000 = 0.9119
Epoch 17: 9260 / 10000 = 0.926


<ipython-input-2-e278208ea042>:2: RuntimeWarning: overflow encountered in exp
  return 1.0/(1.0+np.exp(-z))


Epoch 18: 9185 / 10000 = 0.9185
Epoch 19: 9175 / 10000 = 0.9175
Epoch 20: 9173 / 10000 = 0.9173
Epoch 21: 9149 / 10000 = 0.9149
Epoch 22: 9107 / 10000 = 0.9107
Epoch 23: 9212 / 10000 = 0.9212
Epoch 24: 9249 / 10000 = 0.9249
Epoch 25: 9293 / 10000 = 0.9293
Epoch 26: 9263 / 10000 = 0.9263
Epoch 27: 9176 / 10000 = 0.9176
Epoch 28: 9164 / 10000 = 0.9164
Epoch 29: 9281 / 10000 = 0.9281


# File "Network 2"

In [18]:
"""""""""""""""""""""""""""""""""""""""
network2.py
Модуль создания и обучения нейронной сети для распознавания рукописных цифр
на основе метода стохастического градиентного спуска для прямой нейронной
сети и стоимостной функции на основе перекрестной энтропии, регуляризации и
улучшеннного способа инициализации весов нейронной сети.
Группа: ИСТб-22-1
ФИО: Моргунов Александр Алексеевич
"""""""""""""""""""""""""""""""""""""""
#### Библиотеки
# Стандартные библиотеки
import json # библиотека для кодирования/декодирования данных/объектов Python
import random # библиотека функций для генерации случайных значений
import sys # библиотека для работы с переменными и функциями, имеющими отношение к интерпретатору и его окружению
# Сторонние библиотеки
import numpy as np # библиотека функций для работы с матрицами

""" ---Раздел описаний--- """

""" -- Определение стоимостных функции --"""
class QuadraticCost(object): # Определение среднеквадратичной стоимостной функции
    @staticmethod
    def fn(a, y): # Cтоимостная функция
        return 0.5*np.linalg.norm(a-y)**2
    @staticmethod
    def delta(z, a, y): # Мера влияния нейронов выходного слоя на величину ошибки
        return (a-y) * sigmoid_prime(z)
""" -- Конец описания стоимостной функции -- """
""" -- Описание класса стоимостной функции на основе перекрестной энтропии -- """
class CrossEntropyCost(object): # Определение стоимостной функции на основе перекрестной энтропии
    @staticmethod
    def fn(a, y): # Cтоимостная функция
        return np.sum(np.nan_to_num(-y*np.log(a)-(1-y)*np.log(1-a)))
    @staticmethod
    def delta(z, a, y): # Мера влияния нейронов выходного слоя на величину ошибки
        return (a-y)
""" -- Конец описания класса стоимостной функции на основе перекрестной энтропии -- """
""" --Описание класса Network-- """
class Network2(object):
    def __init__( # конструктор класса
    self # указатель на объект класса
    , sizes # список размеров слоев нейронной сети
    , cost=CrossEntropyCost # стоимостная функция (по умолчанию будет использоваться функция перекрестной энтропии)
    ):
        self.num_layers = len(sizes) # задаем количество слоев нейронной сети
        self.sizes = sizes # задаем список размеров слоев нейронной сети
        self.default_weight_initializer() # метод инициализации начальных весов связей и смещений по умолчанию
        self.cost=cost # задаем стоимостную функцию
    def default_weight_initializer(self): # метод инициализации начальных весов связей и смещений
        self.biases = [np.random.randn(y, 1) for y in self.sizes[1:]] # задаем случайные начальные смещения
        self.weights = [np.random.randn(y, x)/np.sqrt(x) for x, y in zip(self.sizes[:-1], self.sizes[1:])] # задаем случайные начальные веса связей
    def large_weight_initializer(self):
        self.biases = [np.random.randn(y, 1) for y in self.sizes[1:]] # задаем случайные начальные смещения
        self.weights = [np.random.randn(y, x) for x, y in zip(self.sizes[:-1], self.sizes[1:])] # задаем случайные начальные веса
    def feedforward(self, a):
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
        return a

    def SGD(self, training_data, epochs, mini_batch_size, eta,
            lmbda=0.0  # параметр сглаживания L2-регуляризации
            , evaluation_data=None  # оценочная выборка
            , monitor_evaluation_cost=False
            # флаг вывода на экран информации о значении стоимостной функции в процессе обучения, рассчитанном на оценочной выборке
            , monitor_evaluation_accuracy=False
            # флаг вывода на экран информации о достигнутом прогрессе в обучении, рассчитанном на оценочной выборке
            , monitor_training_cost=False
            # флаг вывода на экран информации о значении стоимостной функции в процессе обучения, рассчитанном на обучающей выборке
            , monitor_training_accuracy=False
            # флаг вывода на экран информации о достигнутом прогрессе в обучении, рассчитанном на обучающей выборке
            ):
        if evaluation_data:
            evaluation_data = list(evaluation_data)
            n_data = len(evaluation_data)
        training_data = list(training_data)
        n = len(training_data)
        evaluation_cost, evaluation_accuracy = [], []
        training_cost, training_accuracy = [], []
        for j in range(epochs):
            random.shuffle(training_data)
            mini_batches = [training_data[k:k + mini_batch_size] for k in range(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta, lmbda, len(training_data))
            print("Epoch %s training complete" % j)
            if monitor_training_cost:
                cost = self.total_cost(training_data, lmbda)
                training_cost.append(cost)
                print("--Cost on training data: {}".format(cost))
            if monitor_training_accuracy:
                accuracy = self.accuracy(training_data, convert=True)
                training_accuracy.append(accuracy)
                print("--Accuracy on training data: {} / {}".format(accuracy, n))
            if monitor_evaluation_cost:
                cost = self.total_cost(evaluation_data, lmbda, convert=True)
                evaluation_cost.append(cost)
                print("--Cost on evaluation data: {}".format(cost))
            if monitor_evaluation_accuracy:
                accuracy = self.accuracy(evaluation_data)
                evaluation_accuracy.append(accuracy)
                print("--Accuracy on evaluation data: {} / {}".format(self.accuracy(evaluation_data), n_data))
            print
        return evaluation_cost, evaluation_accuracy, training_cost, training_accuracy
    def update_mini_batch( # Шаг градиентного спуска
            self # указатель на объект класса
            , mini_batch # подвыборка
            , eta # скорость обучения
            , lmbda # параметр сглаживания L2-регуляризации
            , n #
            ):
        nabla_b = [np.zeros(b.shape) for b in self.biases] # список градиентов dC/db для каждого слоя (первоначально заполняются нулями)
        nabla_w = [np.zeros(w.shape) for w in self.weights] # список градиентов dC/dw для каждого слоя (первоначально заполняются нулями)
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y) # послойно вычисляем градиенты dC/db и dC/dw для текущего прецедента (x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)] # суммируем градиенты dC/db для различных прецедентов текущей подвыборки
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)] # суммируем градиенты dC/dw для различных прецедентов текущей подвыборки
        self.weights = [(1-eta*(lmbda/n))*w-(eta/len(mini_batch))*nw for w, nw in zip(self.weights, nabla_w)] # обновляем все веса w нейронной сети
        self.biases = [b-(eta/len(mini_batch))*nb for b, nb in zip(self.biases, nabla_b)] # обновляем все смещения b нейронной сети
    def backprop(# Алгоритм обратного распространения
            self # Указатель на объект класса
            , x # Вектор входных сигналов
            , y # Ожидаемый вектор выходных сигналов
            ):
        nabla_b = [np.zeros(b.shape) for b in self.biases] # список градиентов dC/db для каждого слоя (первоначально заполняются нулями)
        nabla_w = [np.zeros(w.shape) for w in self.weights] # список градиентов dC/dw для каждого слоя (первоначально заполняются нулями)

        # Определение переменных
        activation = x # Выходные сигналы слоя (первоначально соответствует выходным сигналам 1-го слоя или входным сигналам сети)
        activations = [x] # Список выходных сигналов по всем слоям (первоначально содержит только выходные сигналы 1-го слоя)
        zs = [] # Список активационных потенциалов по всем слоям (первоначально пуст)

        # Прямое распространение
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b # Считаем активационные потенциалы текущего слоя
            zs.append(z) # Добавляем элемент (активационные потенциалы слоя) в конец списка
            activation = sigmoid(z) # Считаем выходные сигналы текущего слоя, применяя сигмоидальную функцию активации к активационным потенциалам слоя
            activations.append(activation) # Добавляем элемент (выходные сигналы слоя) в конец списка
        # Обратное распространение
        delta = (self.cost).delta(zs[-1], activations[-1], y) # Считаем меру влияния нейронов выходного слоя L на величину ошибки (BP1)
        nabla_b[-1] = delta # Градиент dC/db для слоя L (BP3)
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())# Градиент dC/dw для слоя L (BP4)
        for l in range(2, self.num_layers):
            z = zs[-l] # Активационные потенциалы l-го слоя (двигаемся по списку справа налево)
            sp = sigmoid_prime(z) # Считаем сигмоидальную функцию от активационных потенциалов l-го слоя
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp # Считаем меру влияния нейронов l-го слоя на величину ошибки (BP2)
            nabla_b[-l] = delta # Градиент dC/db для l-го слоя (BP3)
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w) # Градиент dC/dw для l-го слоя (BP4)
    def accuracy(# Оценка прогресса в обучении
         self # Указатель на объект класса
         , data # Набор данных (выборка)
         , convert=False # Признак необходимости изменять формат представления результата работы нейронной сети
         ):
         if convert:
            results = [(np.argmax(self.feedforward(x)), np.argmax(y)) for (x, y) in data]
         else:
            results = [(np.argmax(self.feedforward(x)), y) for (x, y) in data]
         return sum(int(x == y) for (x, y) in results)
    def total_cost(# Значение функции потерь по всей выборке
         self # Указатель на объект класса
         , data # Набор данных (выборка)
         , lmbda # Параметр сглаживания L2-регуляризации
         , convert=False # Признак необходимости изменять формат представления результата работы нейронной сети
         ):
         cost = 0.0
         data = list(data)
         for x, y in data:
            a = self.feedforward(x)
            if convert: y = vectorized_result(y)
            cost += self.cost.fn(a, y)/len(data)
         cost += 0.5*(lmbda/len(data))*sum(np.linalg.norm(w)**2 for w in self.weights)
         return cost
    def save(self, filename): # Запись нейронной сети в файл
         data = {"sizes": self.sizes,
                "weights": [w.tolist() for w in self.weights],
                "biases": [b.tolist() for b in self.biases],
                "cost": str(self.cost.__name__)}
         f = open(filename, "w")
         json.dump(data, f)
         f.close()

""" -- Конец описания класса Network -- """
def load(filename): # Загрузка нейронной сети из файла

    f = open(filename, "r")
    data = json.load(f)
    f.close()
    cost = getattr(sys.modules[__name__], data["cost"])
    net = Network(data["sizes"], cost=cost)
    net.weights = [np.array(w) for w in data["weights"]]
    net.biases = [np.array(b) for b in data["biases"]]
    return net
def sigmoid(z): # Определение сигмоидальной функции активации
    return 1.0/(1.0+np.exp(-z))
def sigmoid_prime(z): # Производная сигмоидальной функции
    return sigmoid(z)*(1-sigmoid(z))
def vectorized_result(j):
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e

""" --- Конец раздела описаний--- """

' --- Конец раздела описаний--- '

# Interpretator

In [19]:
training_data, validation_data, test_data = load_data_wrapper()

In [20]:
net = Network2([784, 30, 10], cost=CrossEntropyCost)

In [21]:
net.SGD(training_data, 30, 10, 0.5, lmbda = 5.0,evaluation_data=validation_data, monitor_evaluation_accuracy=True, monitor_evaluation_cost=True, monitor_training_accuracy=True, monitor_training_cost=True)

Epoch 0 training complete
--Cost on training data: 0.463420752251909
--Accuracy on training data: 47103 / 50000
--Cost on evaluation data: 0.7591302633617236
--Accuracy on evaluation data: 9445 / 10000
Epoch 1 training complete
--Cost on training data: 0.4194434980167929
--Accuracy on training data: 47701 / 50000
--Cost on evaluation data: 0.8312544650908236
--Accuracy on evaluation data: 9495 / 10000
Epoch 2 training complete
--Cost on training data: 0.41198150874197426
--Accuracy on training data: 47855 / 50000
--Cost on evaluation data: 0.865935508737345
--Accuracy on evaluation data: 9558 / 10000
Epoch 3 training complete
--Cost on training data: 0.39198199314506665
--Accuracy on training data: 48082 / 50000
--Cost on evaluation data: 0.8803700808712751
--Accuracy on evaluation data: 9589 / 10000
Epoch 4 training complete
--Cost on training data: 0.3944074126739984
--Accuracy on training data: 48061 / 50000
--Cost on evaluation data: 0.9041516138840344
--Accuracy on evaluation data

([0.7591302633617236,
  0.8312544650908236,
  0.865935508737345,
  0.8803700808712751,
  0.9041516138840344,
  0.9043646882267737,
  0.9653866630444148,
  0.9171035996957538,
  0.9501967334167409,
  0.9177882997816438,
  0.942083814700518,
  0.9326628553167097,
  0.9376210971151016,
  0.9417219160552714,
  0.9583067826095925,
  0.9697314444358518,
  0.9356510906489506,
  0.9446781840355303,
  0.9370497756031929,
  0.9628887277197204,
  0.9534697715495775,
  0.9332354878949352,
  0.9457037525470118,
  0.9548897703766235,
  0.9489669571804961,
  0.9370657386464198,
  0.979625681467867,
  0.9523932331235556,
  0.9441994152192288,
  0.9442486609386401],
 [9445,
  9495,
  9558,
  9589,
  9584,
  9604,
  9536,
  9611,
  9541,
  9627,
  9587,
  9619,
  9610,
  9598,
  9556,
  9567,
  9601,
  9606,
  9626,
  9579,
  9576,
  9646,
  9639,
  9586,
  9606,
  9636,
  9538,
  9610,
  9613,
  9624],
 [0.463420752251909,
  0.4194434980167929,
  0.41198150874197426,
  0.39198199314506665,
  0.39440741